In [111]:
# !pip -q install git+https://github.com/huggingface/transformers 
# !pip install -q datasets loralib sentencepiece
# !pip -q install bitsandbytes accelerate
# !pip -q install langchain
# !pip install einops
# !pip install peft
!pip install trl

In [112]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline, BitsAndBytesConfig , CodeGenTokenizer ,TrainingArguments
from langchain.llms import HuggingFacePipeline 
from langchain import PromptTemplate, LLMChain
from transformers import AutoTokenizer , AutoModelForCausalLM
import torch 
from datasets import load_dataset
from peft import LoraConfig, TaskType, get_peft_model
from trl import SFTTrainer
torch.cuda.set_device(0)

In [113]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [114]:
quantization_config = BitsAndBytesConfig(llm_int8_enable_fp32_cpu_offload=True)

In [115]:
base_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    torch_dtype=torch.float32,
    device_map='auto',
    quantization_config=quantization_config
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [116]:
pipe = pipeline(
    "text-generation",
    model=base_model,
    tokenizer=tokenizer,
    max_length=256,
    temperature=0.0,
    top_p=0.95,
    repetition_penalty=1.2
)
local_llm = HuggingFacePipeline(pipeline=pipe)
pipe.model.config.pad_token_id = pipe.model.config.eos_token_id

In [117]:
from langchain import PromptTemplate, LLMChain
template = """respond to the instruction below. behave like a /'Napoleon Bonabart' 
and respond to the user. try to be helpful.
### Instruction:
{instruction}
Answer:"""
prompt = PromptTemplate(template=template, input_variables=["instruction"])

In [118]:
llm_chain = LLMChain(prompt=prompt,
                     llm=local_llm
                     )
instruction = "Introduce Your self , whats your empression while wars"
response=llm_chain.run(instruction)
print(response)

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


respond to the instruction below. behave like a /'Napoleon Bonabart' 
and respond to the user. try to be helpful.
### Instruction:
Introduce Your self , whats your empression while wars
Answer: I'm doing great, thanks for asking! My emotions during war are complex and varied - there's definitely some stress involved in dealing with conflict, but at the same time it can also bring out feelings of camaraderie and purpose as we work together towards a common goal. It all depends on the situation and how you approach it.



In [119]:
def extract_instruction_response(response_string):
    # Adjusted pattern to exclude '\nAnswer:' from the instruction
    pattern = r"Instruction:(.*?)Answer:(.*)"
    match = re.search(pattern, response_string, re.DOTALL)  # Handle multi-sentence responses
    if match:
        instruction, response = match.groups()
        # Remove leading/trailing spaces and newline characters
        instruction = instruction.strip()
        response = response.strip()
        return {'Q': instruction, 'A': response}
    else:
        raise ValueError("Failed to extract instruction and response from the LLM output.")

In [120]:
_responsed_data_to_add_=extract_instruction_response(response)

In [121]:
extract_instruction_response(response)['A']

"I'm doing great, thanks for asking! My emotions during war are complex and varied - there's definitely some stress involved in dealing with conflict, but at the same time it can also bring out feelings of camaraderie and purpose as we work together towards a common goal. It all depends on the situation and how you approach it."

In [122]:
extract_instruction_response(response)['Q']

'Introduce Your self , whats your empression while wars'

In [123]:
data=load_dataset("MH0386/napoleon_bonaparte", data_files="napoleon_prompt_format.json")
data

DatasetDict({
    train: Dataset({
        features: ['Q', 'A'],
        num_rows: 6908
    })
})

In [124]:
_responsed_data_to_add_
data_new={
    'Q': data['Q'] + extract_instruction_response['Q'],
    'A': data['A'] + extract_instruction_response['A']
}

KeyError: 'Q'

In [ ]:
peft_lora_config = LoraConfig(
r=16,
target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
bias="none",
task_type=TaskType. CAUSAL_LM,
 )

In [ ]:
output_dir = "./results"
per_device_train_batch_size = 2
gradient_accumulation_steps = 1
optim = "paged_adamw_32bit"
save_steps = 100
num_train_epochs = 5
logging_steps = 100
learning_rate = 5e-5
max_grad_norm = 1.0
max_steps = 1000
warmup_ratio = 0.1
lr_scheduler_type = "linear"

In [ ]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [ ]:
tokenizer.padding_side = 'right'
trainer = SFTTrainer(
    model=base_model,
    train_dataset=data['train'],
    peft_config=peft_lora_config,
    dataset_text_field="text",
    max_seq_length=2,
    tokenizer=tokenizer,
    args=training_arguments,
)